The main points provided for Non-Learning Baseline Readers:
---
1.   The Requirements
2.   ARCD Evaluation
3.   Arabic SQuAD Evaluation
4.   AAQAD Evaluation
5.   View data from AAQAD







# 1.The requirements #
---
We use end to end systems [SOQAL project](https://https://github.com/husseinmozannar/SOQAL) , you will see the requirements we need in details .
- clone project then install requirements.txt then download fasttext file .


In [0]:
 !git clone https://github.com/husseinmozannar/SOQAL.git



Cloning into 'SOQAL'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 92 (delta 11), reused 90 (delta 9), pack-reused 0
Unpacking objects: 100% (92/92), done.


In [0]:
!pip install bottle
!pip install -r requirements.txt

import nltk
nltk.download('punkt')
nltk.download('stopwords')

     |████████████████████████████████| 92kB 2.8MB/s 
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
cd /content/SOQAL/

/content/SOQAL


In [0]:
#for fasttext model
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.af.300.vec.gz

--2019-11-19 15:55:43--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.af.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.6.166, 104.20.22.166, 2606:4700:10::6814:6a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.6.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 525932308 (502M) [binary/octet-stream]
Saving to: ‘cc.af.300.vec.gz’

cc.af.300.vec.gz    100%[===================>] 501.57M  21.9MB/s    in 23s     

2019-11-19 15:56:07 (21.3 MB/s) - ‘cc.af.300.vec.gz’ saved [525932308/525932308]



In [0]:
!gunzip cc.af.300.vec.gz

# 2. ARCD Evaluation

The first arabic data provided in SOQAL 




In [0]:
cd /content/SOQAL/baselines_reading

/content/SOQAL/baselines_reading


In [0]:
!python evaluate_baselines.py  -e '/content/SOQAL/cc.af.300.vec' 

Evaluating the Random Reader ...
{"exact_match": 0.14336917562724014, "f1": 9.02745261606619, "exact_sentence": 52.75985663082437, "inclusion": 98.9247311827957, "random": 30.60931899641577}
Evaluating the TF-IDF Reader ...
{'exact_match': 0.21505376344086022, 'f1': 5.594376664685267, 'exact_sentence': 75.3405017921147, 'inclusion': 95.05376344086021, 'random': 29.892473118279568}
Evaluating the Sliding Window Distance Based Baseline ...


# 3.Arabic SQuAD Evaluation



The second Arabic data provided in SOQAL. We add the path in evaluate_baselines python file.


```
squadArabic_path = "../data/Arabic-SQuAD.json"

```



In [0]:
cd /content/SOQAL/data

/content/SOQAL/data


In [0]:
%%writefile /content/SOQAL/baselines_reading/evaluate_baselines.py
from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys
import nltk
from random import randint
import argparse
squadArabic_path = "../data/Arabic-SQuAD.json"

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def evaluate(dataset, reader):
    f1 = exact_match = total = exact_sentence = inclusion = random = 0
    for article in dataset:
        for paragraph in article['paragraphs']:
            for qa in paragraph['qas']:
               # if qa["is_impossible"]:
                   # continue
                total += 1
                ground_truths = list(map(lambda x: x['text'], qa['answers']))
                prediction = reader.read(paragraph['context'], qa['question'])
                sents = nltk.sent_tokenize(paragraph['context'])
                indx_g = -1
                indx_p = -1
                i = 0
                for sent in sents:
                    if sent.find(ground_truths[0]) != -1:
                        indx_g = i
                    if sent.find(prediction) != -1:
                        indx_p = i
                    i += 1
                test = randint(0, i)
                if test == indx_g:
                    random += 1
                if prediction.find(ground_truths[0]) != -1 or ground_truths[0].find(prediction):
                    inclusion += 1
                if indx_g == indx_p and indx_p != -1:
                    exact_sentence += 1
                exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
                f1 += metric_max_over_ground_truths(
                    f1_score, prediction, ground_truths)
    inclusion = 100 * inclusion / total
    random = 100 * random / total
    exact_sentence = 100 * exact_sentence / total
    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1, 'exact_sentence': exact_sentence, 'inclusion': inclusion,
            'random': random}


from slidingwindow_distance import SWDbasline


def evaluate_reader(dataset_path, reader):
    with open(dataset_path) as f:
        dataset = json.load(f)['data']
    print(json.dumps(evaluate(dataset, reader)))


def evaluate_SWD():
    dataset_path =squadArabic_path
    reader = SWDbasline()
    evaluate_reader(dataset_path, reader)


from embedding_match import embeddingReader
import os

sys.path.append(os.path.abspath("../../embedding"))
from fasttext_embedding import fastTextEmbedder


def evaluate_embeddingReader(emb_path):
    embedder = fastTextEmbedder(emb_path)
    reader = embeddingReader(embedder)
    dataset_path =squadArabic_path
    evaluate_reader(dataset_path, reader)


from tfidf_reader import TfidfReader

def evaluate_TfidfReader():
    dataset_path =squadArabic_path
    with open(dataset_path) as f:
        dataset = json.load(f)['data']
    f1 = exact_match = total = exact_sentence = inclusion = random = 0
    for article in dataset:
        for paragraph in article['paragraphs']:
            reader = TfidfReader(paragraph['context'])
            for qa in paragraph['qas']:
               # if qa["is_impossible"]:
                 # continue
                total += 1
                ground_truths = list(map(lambda x: x['text'], qa['answers']))
                prediction = reader.read(paragraph['context'], qa['question'])
                sents = nltk.sent_tokenize(paragraph['context'])
                indx_g = -1
                indx_p = -1
                i = 0
                for sent in sents:
                    if sent.find(ground_truths[0]) != -1:
                        indx_g = i
                    if sent.find(prediction) != -1:
                        indx_p = i
                    i += 1
                test = randint(0, i)
                if test == indx_g:
                    random += 1
                if prediction.find(ground_truths[0]) != -1 or ground_truths[0].find(prediction):
                    inclusion += 1
                if indx_g == indx_p and indx_p != -1:
                    exact_sentence += 1
                exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
                f1 += metric_max_over_ground_truths(
                    f1_score, prediction, ground_truths)
    inclusion = 100 * inclusion / total
    random = 100 * random / total
    exact_sentence = 100 * exact_sentence / total
    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    to_print = {'exact_match': exact_match, 'f1': f1, 'exact_sentence': exact_sentence, 'inclusion': inclusion,
                'random': random}
    print(to_print)


from random_reader import RandomReader


def evaluate_RandomReader():
    reader = RandomReader()
    dataset_path =squadArabic_path
    evaluate_reader(dataset_path, reader)


parser = argparse.ArgumentParser()
parser.add_argument('-e', '--emb', help='Location of fastText embedding .vec file', required=True)


def main():
    args = parser.parse_args()
    print('Evaluating the Random Reader ...')
    evaluate_RandomReader()
    print('Evaluating the TF-IDF Reader ...')
    evaluate_TfidfReader()
    print('Evaluating the Sliding Window Distance Based Baseline ...')
    evaluate_SWD()
    print('Evaluating an embedding based reader using fastText ...')
    evaluate_embeddingReader(args.emb)


if __name__ == "__main__":
    main()


Overwriting /content/SOQAL/baselines_reading/evaluate_baselines.py


In [0]:
cd /content/SOQAL/baselines_reading

/content/SOQAL/baselines_reading


In [0]:
!python evaluate_baselines.py  -e '/content/SOQAL/cc.af.300.vec'

Evaluating the Random Reader ...
{"exact_match": 0.14893264934635114, "f1": 4.924809966107995, "exact_sentence": 28.9674002978653, "inclusion": 99.76418997186828, "random": 18.790335925864635}
Evaluating the TF-IDF Reader ...
{'exact_match': 0.2709746814496111, 'f1': 2.7638244816521103, 'exact_sentence': 54.920982955485684, 'inclusion': 99.19741850074466, 'random': 19.05096806222075}
Evaluating the Sliding Window Distance Based Baseline ...
{"exact_match": 0.01447956313089525, "f1": 6.912161214594479, "exact_sentence": 33.48295548568592, "inclusion": 100.0, "random": 18.825500579182524}
Evaluating an embedding based reader using fastText ...
loading fastText model ...
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for detai

# **4. AAQAD Evaluation**
#This our data provided question with no answers . You can download from [AAQAD](https://drive.google.com/uc?id=1jhUmWb9eHVATqhrWKAXxSE2gqJ53-wk6)

**4.1.   Modified for questions with no answers**
```
  for article in dataset:
        for paragraph in article['paragraphs']:
            for qa in paragraph['qas']:
                if qa['is_impossible']:
                  continue
                total += 1
                ground_truths = list(map(lambda x: x['text'], qa['answers']))
                prediction = reader.read(paragraph['context'], qa['question'])
                sents = nltk.sent_tokenize(paragraph['context'])
                indx_g = -1
                indx_p = -1
                i = 0
```



**4.2. Adding PATH for AAQAD**


```
aaqad_path = "../data/aaqad.json"

```




In [0]:
cd /content/SOQAL/data

/content/SOQAL/data


In [0]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1jhUmWb9eHVATqhrWKAXxSE2gqJ53-wk6 -O aaqad.json

Downloading...
From: https://drive.google.com/uc?id=1jhUmWb9eHVATqhrWKAXxSE2gqJ53-wk6
To: /content/SOQAL/data/aaqad.json
9.94MB [00:00, 154MB/s]


In [0]:
%%writefile /content/SOQAL/baselines_reading/evaluate_baselines.py
from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys
import nltk
from random import randint
import argparse
aaqad_path = "../data/aaqad.json"

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def evaluate(dataset, reader):
    f1 = exact_match = total = exact_sentence = inclusion = random = 0
    for article in dataset:
        for paragraph in article['paragraphs']:
            for qa in paragraph['qas']:
                if qa["is_impossible"]:
                    continue
                total += 1
                ground_truths = list(map(lambda x: x['text'], qa['answers']))
                prediction = reader.read(paragraph['context'], qa['question'])
                sents = nltk.sent_tokenize(paragraph['context'])
                indx_g = -1
                indx_p = -1
                i = 0
                for sent in sents:
                    if sent.find(ground_truths[0]) != -1:
                        indx_g = i
                    if sent.find(prediction) != -1:
                        indx_p = i
                    i += 1
                test = randint(0, i)
                if test == indx_g:
                    random += 1
                if prediction.find(ground_truths[0]) != -1 or ground_truths[0].find(prediction):
                    inclusion += 1
                if indx_g == indx_p and indx_p != -1:
                    exact_sentence += 1
                exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
                f1 += metric_max_over_ground_truths(
                    f1_score, prediction, ground_truths)
    inclusion = 100 * inclusion / total
    random = 100 * random / total
    exact_sentence = 100 * exact_sentence / total
    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1, 'exact_sentence': exact_sentence, 'inclusion': inclusion,
            'random': random}


from slidingwindow_distance import SWDbasline


def evaluate_reader(dataset_path, reader):
    with open(dataset_path) as f:
        dataset = json.load(f)['data']
    print(json.dumps(evaluate(dataset, reader)))


def evaluate_SWD():
    dataset_path = aaqad_path
    reader = SWDbasline()
    evaluate_reader(dataset_path, reader)


from embedding_match import embeddingReader
import os

sys.path.append(os.path.abspath("../../embedding"))
from fasttext_embedding import fastTextEmbedder


def evaluate_embeddingReader(emb_path):
    embedder = fastTextEmbedder(emb_path)
    reader = embeddingReader(embedder)
    dataset_path = aaqad_path
    evaluate_reader(dataset_path, reader)


from tfidf_reader import TfidfReader

def evaluate_TfidfReader():
    dataset_path = aaqad_path
    with open(dataset_path) as f:
        dataset = json.load(f)['data']
    f1 = exact_match = total = exact_sentence = inclusion = random = 0
    for article in dataset:
        for paragraph in article['paragraphs']:
            reader = TfidfReader(paragraph['context'])
            for qa in paragraph['qas']:
                if qa["is_impossible"]:
                  continue
                total += 1
                ground_truths = list(map(lambda x: x['text'], qa['answers']))
                prediction = reader.read(paragraph['context'], qa['question'])
                sents = nltk.sent_tokenize(paragraph['context'])
                indx_g = -1
                indx_p = -1
                i = 0
                for sent in sents:
                    if sent.find(ground_truths[0]) != -1:
                        indx_g = i
                    if sent.find(prediction) != -1:
                        indx_p = i
                    i += 1
                test = randint(0, i)
                if test == indx_g:
                    random += 1
                if prediction.find(ground_truths[0]) != -1 or ground_truths[0].find(prediction):
                    inclusion += 1
                if indx_g == indx_p and indx_p != -1:
                    exact_sentence += 1
                exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
                f1 += metric_max_over_ground_truths(
                    f1_score, prediction, ground_truths)
    inclusion = 100 * inclusion / total
    random = 100 * random / total
    exact_sentence = 100 * exact_sentence / total
    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    to_print = {'exact_match': exact_match, 'f1': f1, 'exact_sentence': exact_sentence, 'inclusion': inclusion,
                'random': random}
    print(to_print)


from random_reader import RandomReader


def evaluate_RandomReader():
    reader = RandomReader()
    dataset_path = aaqad_path
    evaluate_reader(dataset_path, reader)


parser = argparse.ArgumentParser()
parser.add_argument('-e', '--emb', help='Location of fastText embedding .vec file', required=True)


def main():
    args = parser.parse_args()
    print('Evaluating the Random Reader ...')
    evaluate_RandomReader()
    print('Evaluating the TF-IDF Reader ...')
    evaluate_TfidfReader()
    print('Evaluating the Sliding Window Distance Based Baseline ...')
    evaluate_SWD()
    print('Evaluating an embedding based reader using fastText ...')
    evaluate_embeddingReader(args.emb)


if __name__ == "__main__":
    main()


Overwriting /content/SOQAL/baselines_reading/evaluate_baselines.py


In [0]:
cd /content/SOQAL/baselines_reading

/content/SOQAL/baselines_reading


In [0]:
!python evaluate_baselines.py  -e '/content/SOQAL/cc.af.300.vec'

Evaluating the Random Reader ...
{"exact_match": 0.15759061460339696, "f1": 5.825756563341704, "exact_sentence": 40.211871826300126, "inclusion": 99.67606373664857, "random": 24.01505865872877}
Evaluating the TF-IDF Reader ...
{'exact_match': 1.1031343022237787, 'f1': 3.976363339056766, 'exact_sentence': 53.73839957975836, 'inclusion': 98.00385221502364, 'random': 23.262125722290317}
Evaluating the Sliding Window Distance Based Baseline ...
{"exact_match": 0.2188758536158291, "f1": 9.199589736775097, "exact_sentence": 43.74890562073192, "inclusion": 100.0, "random": 23.42847137103835}
Evaluating an embedding based reader using fastText ...
loading fastText model ...
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details

# **5.View data from AAQAD**

In [0]:
import io
import json
import textwrap

def saveJSON(path, data):
  jsonVar = json.dumps(data, ensure_ascii=False)
  json_file = io.open(path, mode="w", encoding="utf-8")
  json_file.write(jsonVar)
  json_file.close()
def loadJSON(path):
  json_file = io.open(path, mode="r", encoding="utf-8")
  return json.load(json_file)
def printWrapped(text, lineWidth):
  wrapper = textwrap.TextWrapper(width=lineWidth) 
  word_list = wrapper.wrap(text=text)  
  for element in word_list: 
    print(element) 

In [0]:
cd /content/SOQAL/data_helpers

/content/SOQAL/data_helpers


In [0]:
!python view_data.py -d ../data/arcd.json -v 1

######################################################
######################################################
######################################################
Article:
بيونسيه
######################################################
Paragraph:
بيونسي جيزيل نولز-كارتر (من مواليد 4 سبتمبر، 1981)، المعروفة باسم بيونسي. ولدت ونشأت في هيوستن بولاية تكساس، هي مغنية وممثلة أميركية حائزة على 23 جائزة غرامي.غنت في مسابقات غناء ورقص مختلفة عندما كانت طفلة، أصبحت مشهورة في أواخر التسعينات كمغنية آر أند بي (رئيسية) للفرقة الغنائية النسائية دستنيز تشايلد. والتي أديرت من قِبل والدها ماثيو نولز، وأصبحت الفرقة واحدة من الأكثر مبيعاً في العالم من الفرق النسائية على الإطلاق. وقد شهد إنفصال الفرقة المؤقت صدور ألبوم بيونسي الأول  Dangerously in Love دانجيروسلي إن لوف  (2003)، والذي أنشأها بأن تكون فنان منفرد ناجح في العالم؛ بيعت منه 16 مليون نسخة، حصل على خمسة جوائز غرامي وتضمن الأغاني التي وصلت إلى قمة الرسم البياني الأمريكي بيلبورد هوت 100 "كريزي إن لوف" و"بيبي بوي". 

###########################
Qu

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

